In [5]:
# Decorators
from functools import wraps
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import numpy as np

def my_logger(orig_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(orig_func.__name__), level=logging.INFO)

    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        logging.info('Ran with args: {}, and kwargs: {}'.format(args, kwargs))
        #logging.info('')
        return orig_func(*args, **kwargs)

    return wrapper


def my_timer(orig_func):
    import time

    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time() - t1
        print('{} ran in: {} sec'.format(orig_func.__name__, t2))
        return result

    return wrapper

In [6]:
def download():
    from sklearn.datasets import fetch_mldata
    mnist = fetch_mldata('MNIST original')
    X = mnist.data.astype('float64')
    y = mnist.target
    return (X, y) 

In [7]:
class Normalize(object): 
    def normalize(self, X_train, X_test):
        self.scaler = MinMaxScaler()
        X_train = self.scaler.fit_transform(X_train)
        X_test  = self.scaler.transform(X_test)
        return (X_train, X_test) 
    
    def inverse(self, X_train, X_val, X_test):
        X_train = self.scaler.inverse_transform(X_train)
        X_test  = self.scaler.inverse_transform(X_test)
        return (X_train, X_test)   

def split(X,y, splitRatio):
    X_train = X[:splitRatio]
    y_train = y[:splitRatio]
    X_test = X[splitRatio:]
    y_test = y[splitRatio:]
    return (X_train, y_train, X_test, y_test)  

In [8]:
class TheAlgorithm(object):
  
    @my_logger
    @my_timer
    def __init__(self, X_train, y_train, X_test, y_test):  
      self.X_train, self.y_train, self.X_test, self.y_test = X_train, y_train, X_test, y_test    
        
    @my_logger
    @my_timer
    def fit(self): 
        normalizer = Normalize()
        self.X_train, self.X_test = normalizer.normalize(self.X_train, self.X_test)   
        train_samples = self.X_train.shape[0]
        self.classifier = LogisticRegression(
            C=50. / train_samples,
            multi_class='multinomial',
            penalty='l1',
            solver='saga',
            tol=0.1,
            class_weight='balanced',
            )
        self.classifier.fit(self.X_train, self.y_train)
        self.train_y_predicted = self.classifier.predict(self.X_train)
        self.train_accuracy = np.mean(self.train_y_predicted.ravel() == self.y_train.ravel()) * 100
        self.train_confusion_matrix = confusion_matrix(self.y_train, self.train_y_predicted)        
        return self.train_accuracy
    
    @my_logger
    @my_timer
    def predict(self):
        self.test_y_predicted = self.classifier.predict(self.X_test) 
        self.test_accuracy = np.mean(self.test_y_predicted.ravel() == self.y_test.ravel()) * 100 
        self.test_confusion_matrix = confusion_matrix(self.y_test, self.test_y_predicted)        
        self.report = classification_report(self.y_test, self.test_y_predicted)
        print("Classification report for classifier:\n %s\n" % (self.report))
        return self.test_accuracy

In [9]:
#The solution
if __name__ == '__main__': 
  
        X,y = download()
        print ('MNIST:', X.shape, y.shape)
  
        splitRatio = 60000
        X_train, y_train, X_test, y_test = split(X,y,splitRatio) 

        np.random.seed(31337)
        ta = TheAlgorithm(X_train, y_train, X_test, y_test)
        train_accuracy = ta.fit()
        print()
        print('Train Accuracy:', train_accuracy,'\n') 
        print("Train confusion matrix:\n%s\n" % ta.train_confusion_matrix)
  
        test_accuracy = ta.predict()
        print()
        print('Test Accuracy:', test_accuracy,'\n') 
        print("Test confusion matrix:\n%s\n" % ta.test_confusion_matrix)

MNIST: (70000, 784) (70000,)
__init__ ran in: 5.245208740234375e-06 sec
fit ran in: 17.269278049468994 sec

Train Accuracy: 72.92166666666667 

Train confusion matrix:
[[5447    5   40   31   49   16  198   50   81    6]
 [   3 6440  127   54    3   29   25   36   24    1]
 [ 297  420 3824  163  256   19  622  186  121   50]
 [ 124  221  255 4566   54  251   97  129  275  159]
 [ 104  128   26   54 4546  342  206  133   96  207]
 [ 399  200  109 1081  416 2227  289  363  228  109]
 [ 173   89  112   55  156  229 5034   25   45    0]
 [ 213  192  205   39  160   17   26 5058   60  295]
 [  67  690  202  677   73  188  347   39 3437  131]
 [ 164  162   63  290  669  279  122  735  291 3174]]

Classification report for classifier:
              precision    recall  f1-score   support

        0.0       0.79      0.94      0.86       980
        1.0       0.77      0.96      0.85      1135
        2.0       0.78      0.65      0.71      1032
        3.0       0.66      0.76      0.71      